In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso

In [4]:
data_bin = pd.read_csv('./csv/binary_classification_data.csv')

In [5]:
# Check for missing values
missing_values = data_bin.isnull().sum()

# Display the number of missing values per column
missing_values

 Mean of the integrated profile                  0
 Standard deviation of the integrated profile    0
 Excess kurtosis of the integrated profile       0
 Skewness of the integrated profile              0
 Mean of the DM-SNR curve                        0
 Standard deviation of the DM-SNR curve          0
 Excess kurtosis of the DM-SNR curve             0
 Skewness of the DM-SNR curve                    0
target_class                                     0
dtype: int64

In [6]:
# Display basic statistics of the dataset
data_bin.describe()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
count,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000
mean,111.079968,46.549532,0.477857,1.770279,12.614400,26.326515,8.303556,104.857709,0.091574
std,25.652935,6.843189,1.064040,6.167913,29.472897,19.470572,4.506092,106.514540,0.288432
min,5.812500,24.772042,-1.876011,-1.791886,0.213211,7.370432,-3.139270,-1.976976,0.000000
25%,100.929688,42.376018,0.027098,-0.188572,1.923077,14.437332,5.781506,34.960504,0.000000
50%,115.078125,46.947479,0.223240,0.198710,2.801839,18.461316,8.433515,83.064556,0.000000
75%,127.085938,51.023202,0.473325,0.927783,5.464256,28.428104,10.702959,139.309330,0.000000
max,192.617188,98.778911,8.069522,68.101622,223.392141,110.642211,34.539844,1191.000837,1.000000


StandardScaler를 사용하여 각 특성을 평균이 0이고 표준 편차가 1이 되도록 변환.   
주의할 점은, 스케일러는 학습 데이터에만 적합해야 하며, 이 스케일러를 사용하여 테스트 데이터도 변환해야 한다는 것입니다.   
이렇게 하는 이유는 테스트 데이터는 학습 과정에서 보이지 않아야 하며, 실제 모델이 처음 보는 데이터에 대한 성능을 정확하게 측정하려면 이와 같은 절차가 필요하기 때문입니다

In [7]:
# Define the features and the target
X = data_bin.drop("target_class", axis=1)
y = data_bin["target_class"]

# Initialize a new StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features and transform
X_scaled = scaler.fit_transform(X)

# Convert the scaled features into a DataFrame
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Display the first few rows of the scaled features
X_scaled.head()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve
0,1.149317,1.334832,-0.669570,-0.400459,-0.319440,-0.370625,-0.072798,-0.287438
1,-0.334168,1.802265,-0.011785,-0.370535,-0.371102,-0.588924,0.504427,0.211581
2,-0.314372,-1.053322,-0.145233,-0.116593,-0.322107,-0.235328,-0.125996,-0.391373
3,1.000694,1.553254,-0.513409,-0.390178,-0.304404,-0.275666,-0.312265,-0.481300
4,-0.871402,-0.858879,0.115609,-0.104866,-0.388010,-0.763111,1.324026,1.386794


각 특성이 평균 0과 표준 편차 1을 가지도록 스케일링되었습니다. 이 데이터를 이용해서 이진 분류 모델을 학습시킬 준비가 되었습니다.

다음 단계는 이 데이터를 학습 데이터셋과 테스트 데이터셋으로 분할하는 것입니다. 이를 위해서는 일반적으로 sklearn의 **`train_test_split`** 함수를 사용합니다. 분할 비율을 지정해야 하는데, 일반적으로 전체 데이터셋의 약 70~80%를 학습에 사용하고, 나머지를 테스트에 사용합니다. 이렇게 분할해보겠습니다.

In [ ]:
# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Check the shapes of the resulting datasets
X_train.shape, X_test.shape, y_train.shape, y_test.shape